In [172]:
from collections import defaultdict
from itertools import combinations


def get_l1_itemset(transactions, minsupp):
    k1_itemset = []
    item_count = defaultdict(int)
    freq_itemset = dict()
    n_transaction = len(transactions)
    for transaction in transactions:
        for item in transaction:
            item_count[(item,)] = item_count[(item,)] + 1
    for key, value in item_count.items():
        supp = value / n_transaction
        if supp >= minsupp:
            k1_itemset.append([key[0]])
            freq_itemset[key] = supp
    return sorted(k1_itemset), item_count, freq_itemset


def get_lk_candidate(pruned_itemset, transactions, minsupp):
    k_itemset = []
    item_count = {}
    freq_itemset = {}
    k = len(pruned_itemset[0])
    for pruned in pruned_itemset:
        item_count[tuple(pruned)] = 0
    n_transaction = len(transactions)
    for transaction in transactions:
        if len(transaction) >= k:
            for comb in combinations(transaction, k):
                if comb in item_count:
                    item_count[comb] = item_count[comb] + 1
    for key, value in item_count.items():
        supp = value / n_transaction
        if supp >= minsupp:
            k_itemset.append(sorted(list(key)))
            freq_itemset[key] = supp
    return sorted(k_itemset), item_count, freq_itemset


def apriori_gen(k_itemset):
    candidate_itemset = set()
    for item1, item2 in combinations(k_itemset, 2):
        if item1[:-1] == item2[:-1]:
            if item1[-1] < item2[-1]:
                candidate_itemset.add(tuple(item1 + [item2[-1]]))
    candidate_itemset = list(candidate_itemset)
    return sorted([list(x) for x in candidate_itemset])


def prune(candidate_itemset, prev_itemset):
    k = len(prev_itemset[0]) + 1
    pruned_itemset = []
    for itemset in candidate_itemset:
        valid = True
        for combination in combinations(itemset, k-1):
            if list(combination) not in prev_itemset:
                valid = False
        if valid:
            pruned_itemset.append(itemset)
    return pruned_itemset


def get_assoc_itemsets(lk_itemsets, minconf, item_count_dict, freq_itemsets, max_k):
    assoc_itemsets = dict()
    all_assoc_itemsets = dict()
    for k in range(2, max_k+1):
        assoc_k_itemset = set()
        k_itemset = lk_itemsets[k]
        for itemset in k_itemset:
            for i in range(k):
                full = tuple(itemset)
                left, right = itemset[:i] + itemset[i+1:], itemset[i]
                assoc_k_itemset.add((tuple(full), tuple(left), right))
        for full, left, right in assoc_k_itemset:
            conf = item_count_dict[full] / item_count_dict[left]
            supp = freq_itemsets[full]
            all_assoc_itemsets[(left, right)] = conf
            if conf >= minconf:
                assoc_itemsets[(left, right)] = (conf, supp)
    return assoc_itemsets, all_assoc_itemsets


def get_frequent_itemsets(transactions, minsupp, minconf):
    freq_itemsets = dict()
    lk_itemsets = dict()
    item_count_dict = dict()
    k = 1
    k1_itemset, item_count, freq_itemset = get_l1_itemset(transactions, minsupp)
    if len(k1_itemset) == 0:
        raise ValueError("None of the k1-itemset is frequent")
    item_count_dict.update(item_count)
    freq_itemsets.update(freq_itemset)
    lk_itemsets[k] = k1_itemset
    prev_itemset = k1_itemset
    while len(prev_itemset) > 0:
        candidate_itemset = apriori_gen(prev_itemset)
        if len(candidate_itemset) == 0:
            break
        pruned_itemset = prune(candidate_itemset, prev_itemset)
        if len(pruned_itemset) == 0:
            break
        k_itemset, item_count, freq_itemset = get_lk_candidate(pruned_itemset, transactions, minsupp)
        item_count_dict.update(item_count)
        if len(k_itemset) != 0:
            k = k + 1
            freq_itemsets.update(freq_itemset)
            lk_itemsets[k] = k_itemset
        else:
            break
        prev_itemset = k_itemset
    assoc_itemsets, all_assoc_itemsets = get_assoc_itemsets(lk_itemsets, minconf, item_count_dict, freq_itemsets, k)
    return freq_itemsets, assoc_itemsets, all_assoc_itemsets, lk_itemsets

In [181]:
transactions = [['pen','ink','diary','soap'],
['pen','ink','diary'],
['pen','diary', 'book'],
['pen','ink','soap']]

transactions = [sorted(x) for x in transactions]
minsupp = 0.5
minconf = 0.5

In [182]:
transactions2 = [['pen', 'ink', 'diary', 'soap'], ['pen', 'ink', 'diary'], ['pen', 'diary', 'book'], ['pen', 'ink', 'soap']]

In [189]:
for t in transactions:
    print(','.join(t))

diary,ink,pen,soap
diary,ink,pen
book,diary,pen
ink,pen,soap


In [188]:
transactions2

[['pen', 'ink', 'diary', 'soap'],
 ['pen', 'ink', 'diary'],
 ['pen', 'diary', 'book'],
 ['pen', 'ink', 'soap']]

In [185]:
freq_itemsets, assoc_itemsets, all_assoc_itemsets, lk_itemsets = get_frequent_itemsets(transactions2, minsupp, minconf)

In [186]:
freq_itemsets

{('pen',): 1.0,
 ('ink',): 0.75,
 ('diary',): 0.75,
 ('soap',): 0.5,
 ('ink', 'soap'): 0.5,
 ('pen', 'soap'): 0.5}

In [30]:
freq_itemsets

{('diary',): 0.75,
 ('ink',): 0.75,
 ('pen',): 1.0,
 ('soap',): 0.5,
 ('diary', 'ink'): 0.5,
 ('diary', 'pen'): 0.75,
 ('ink', 'pen'): 0.75,
 ('ink', 'soap'): 0.5,
 ('pen', 'soap'): 0.5,
 ('diary', 'ink', 'pen'): 0.5,
 ('ink', 'pen', 'soap'): 0.5}

In [31]:
assoc_itemsets

{(('soap',), 'ink'): (1.0, 0.5),
 (('pen',), 'ink'): (0.75, 0.75),
 (('diary',), 'pen'): (1.0, 0.75),
 (('ink',), 'diary'): (0.6666666666666666, 0.5),
 (('diary',), 'ink'): (0.6666666666666666, 0.5),
 (('pen',), 'diary'): (0.75, 0.75),
 (('pen',), 'soap'): (0.5, 0.5),
 (('ink',), 'pen'): (1.0, 0.75),
 (('soap',), 'pen'): (1.0, 0.5),
 (('ink',), 'soap'): (0.6666666666666666, 0.5),
 (('diary', 'pen'), 'ink'): (0.6666666666666666, 0.5),
 (('ink', 'pen'), 'diary'): (0.6666666666666666, 0.5),
 (('ink', 'soap'), 'pen'): (1.0, 0.5),
 (('diary', 'ink'), 'pen'): (1.0, 0.5),
 (('pen', 'soap'), 'ink'): (1.0, 0.5),
 (('ink', 'pen'), 'soap'): (0.6666666666666666, 0.5)}

In [35]:
def format_freq_itemsets(freq_itemsets, minsupp):
    print("==Frequent itemsets (min_sup={:.1f}%)".format(minsupp * 100.))
    freq = [(k, v) for k, v in freq_itemsets.items()]
    freq.sort(key = lambda x: (x[1], x[0]), reverse=True)
    for k, v in freq:
        print("[{}], {:.1f}%".format(','.join(k), v * 100.))



def format_assoc_itemsets(assoc_itemsets, minconf):
    print("==High-confidence association rules (min_conf={:.1f}%)".format(minconf * 100.))
    assoc = [(k, v) for k, v in assoc_itemsets.items()]
    assoc.sort(key = lambda x: (x[1][0], x[1][1], x[0]), reverse=True)
    for k, v in assoc:
        print("[{}] => [{}] (Conf: {:.1f}%, Supp: {:.1f}%)".format(','.join(k[0]), k[1], v[0] * 100., v[1] * 100.))

In [36]:
format_freq_itemsets(freq_itemsets, 0.5)

==Frequent itemsets (min_sup=50.0%)
[pen], 100.0%
[ink,pen], 75.0%
[ink], 75.0%
[diary,pen], 75.0%
[diary], 75.0%
[soap], 50.0%
[pen,soap], 50.0%
[ink,soap], 50.0%
[ink,pen,soap], 50.0%
[diary,ink,pen], 50.0%
[diary,ink], 50.0%


In [37]:
format_assoc_itemsets(assoc_itemsets, 0.5)

==High-confidence association rules (min_conf=50.0%)
[ink] => [pen] (Conf: 100.0%, Supp: 75.0%)
[diary] => [pen] (Conf: 100.0%, Supp: 75.0%)
[soap] => [pen] (Conf: 100.0%, Supp: 50.0%)
[soap] => [ink] (Conf: 100.0%, Supp: 50.0%)
[pen,soap] => [ink] (Conf: 100.0%, Supp: 50.0%)
[ink,soap] => [pen] (Conf: 100.0%, Supp: 50.0%)
[diary,ink] => [pen] (Conf: 100.0%, Supp: 50.0%)
[pen] => [ink] (Conf: 75.0%, Supp: 75.0%)
[pen] => [diary] (Conf: 75.0%, Supp: 75.0%)
[ink,pen] => [soap] (Conf: 66.7%, Supp: 50.0%)
[ink,pen] => [diary] (Conf: 66.7%, Supp: 50.0%)
[ink] => [soap] (Conf: 66.7%, Supp: 50.0%)
[ink] => [diary] (Conf: 66.7%, Supp: 50.0%)
[diary,pen] => [ink] (Conf: 66.7%, Supp: 50.0%)
[diary] => [ink] (Conf: 66.7%, Supp: 50.0%)
[pen] => [soap] (Conf: 50.0%, Supp: 50.0%)


In [13]:
%debug

> <ipython-input-10-354d91c8b769>(14)format_assoc_itemsets()
     10     print("==High-confidence association rules (min_conf={:.1f}%)".format(minconf * 100.))
     11     assoc = [(k, v) for k, v in assoc_itemsets.items()]
     12     assoc.sort(key = lambda x: (x[1][0], x[1][1], x[0]))
     13     for k, v in assoc:
---> 14         print("[{}] => [{}] (Conf: {:.1f}%, Supp: {:.1f}%)".format(','.join(k), v[0] * 100., v[1] * 100.))

ipdb> k
(('diary', 'pen'), 'ink')
ipdb> v
(0.6666666666666666, 0.5)
ipdb> q


In [126]:
freq_itemsets = dict()
lk_itemsets = dict()
item_count_dict = dict()
k = 1
k1_itemset, item_count, freq_itemset = get_l1_itemset(transactions, minsupp)
if len(k1_itemset) == 0:
    raise ValueError("None of the k1-itemset is frequent")

In [127]:
print(k1_itemset)
print(item_count)
print(freq_itemset)

[['diary'], ['ink'], ['pen'], ['soap']]
defaultdict(<class 'int'>, {('diary',): 3, ('ink',): 3, ('pen',): 4, ('soap',): 2, ('book',): 1})
{('diary',): 0.75, ('ink',): 0.75, ('pen',): 1.0, ('soap',): 0.5}


In [128]:
item_count_dict.update(item_count)
freq_itemsets.update(freq_itemset)
lk_itemsets[k] = k1_itemset
prev_itemset = k1_itemset

In [130]:
print(item_count_dict)
print(freq_itemsets)
print(lk_itemsets)

{('diary',): 3, ('ink',): 3, ('pen',): 4, ('soap',): 2, ('book',): 1}
{('diary',): 0.75, ('ink',): 0.75, ('pen',): 1.0, ('soap',): 0.5}
{1: [['diary'], ['ink'], ['pen'], ['soap']]}


In [131]:
while len(prev_itemset) > 0:
    candidate_itemset = apriori_gen(prev_itemset)
    if len(candidate_itemset) == 0:
        break
    pruned_itemset = prune(candidate_itemset, prev_itemset)
    if len(pruned_itemset) == 0:
        break
    k_itemset, item_count, freq_itemset = get_lk_candidate(pruned_itemset, transactions, minsupp)
    item_count_dict.update(item_count)
    if len(k_itemset) != 0:
        k = k + 1
        freq_itemsets.update(freq_itemset)
        lk_itemsets[k] = k_itemset
    else:
        break
    prev_itemset = k_itemset

In [132]:
print(item_count_dict)
print(freq_itemsets)
print(lk_itemsets)

{('diary',): 3, ('ink',): 3, ('pen',): 4, ('soap',): 2, ('book',): 1, ('diary', 'ink'): 2, ('diary', 'pen'): 3, ('diary', 'soap'): 1, ('ink', 'pen'): 3, ('ink', 'soap'): 2, ('pen', 'soap'): 2, ('diary', 'ink', 'pen'): 2, ('ink', 'pen', 'soap'): 2}
{('diary',): 0.75, ('ink',): 0.75, ('pen',): 1.0, ('soap',): 0.5, ('diary', 'ink'): 0.5, ('diary', 'pen'): 0.75, ('ink', 'pen'): 0.75, ('ink', 'soap'): 0.5, ('pen', 'soap'): 0.5, ('diary', 'ink', 'pen'): 0.5, ('ink', 'pen', 'soap'): 0.5}
{1: [['diary'], ['ink'], ['pen'], ['soap']], 2: [['diary', 'ink'], ['diary', 'pen'], ['ink', 'pen'], ['ink', 'soap'], ['pen', 'soap']], 3: [['diary', 'ink', 'pen'], ['ink', 'pen', 'soap']]}


In [133]:
assoc_itemsets = get_assoc_itemsets(lk_itemsets, minconf, item_count_dict, k)

In [134]:
assoc_itemsets

({(('ink', 'soap'), 'pen'): 1.0,
  (('ink', 'pen'), 'soap'): 0.6666666666666666,
  (('diary', 'pen'), 'ink'): 0.6666666666666666,
  (('pen', 'soap'), 'ink'): 1.0,
  (('ink', 'pen'), 'diary'): 0.6666666666666666,
  (('diary', 'ink'), 'pen'): 1.0},
 {(('ink', 'soap'), 'pen'): 1.0,
  (('ink', 'pen'), 'soap'): 0.6666666666666666,
  (('diary', 'pen'), 'ink'): 0.6666666666666666,
  (('pen', 'soap'), 'ink'): 1.0,
  (('ink', 'pen'), 'diary'): 0.6666666666666666,
  (('diary', 'ink'), 'pen'): 1.0})

In [136]:
list(assoc_itemsets)

[{(('ink', 'soap'), 'pen'): 1.0,
  (('ink', 'pen'), 'soap'): 0.6666666666666666,
  (('diary', 'pen'), 'ink'): 0.6666666666666666,
  (('pen', 'soap'), 'ink'): 1.0,
  (('ink', 'pen'), 'diary'): 0.6666666666666666,
  (('diary', 'ink'), 'pen'): 1.0},
 {(('ink', 'soap'), 'pen'): 1.0,
  (('ink', 'pen'), 'soap'): 0.6666666666666666,
  (('diary', 'pen'), 'ink'): 0.6666666666666666,
  (('pen', 'soap'), 'ink'): 1.0,
  (('ink', 'pen'), 'diary'): 0.6666666666666666,
  (('diary', 'ink'), 'pen'): 1.0}]

In [ ]:
transactions = [['pen','ink','diary','soap'],
['pen','ink','diary'],
['pen','diary', 'book'],
['pen','ink','soap']]

transactions = [sorted(x) for x in transactions]


In [125]:
minsupp = 0.5
minconf = 0.5

In [52]:
large_itemsets = dict()
item_counts = dict()
k = 1
prev_itemset, item_count = get_l1_itemset(transactions, minsupp)
item_counts.update(item_count)
large_itemsets[k] = prev_itemset

In [53]:
large_itemsets

{1: [['diary'], ['ink'], ['pen'], ['soap']]}

In [54]:
item_counts

{('diary',): 3, ('ink',): 3, ('pen',): 4, ('soap',): 2, ('book',): 1}

In [55]:
candidate_itemset = apriori_gen(prev_itemset)
candidate_itemset

[['diary', 'ink'],
 ['diary', 'pen'],
 ['diary', 'soap'],
 ['ink', 'pen'],
 ['ink', 'soap'],
 ['pen', 'soap']]

In [56]:
pruned_itemset = prune(candidate_itemset, prev_itemset)
pruned_itemset

[['diary', 'ink'],
 ['diary', 'pen'],
 ['diary', 'soap'],
 ['ink', 'pen'],
 ['ink', 'soap'],
 ['pen', 'soap']]

In [57]:
large_itemset, item_count = get_lk_candidate(pruned_itemset, transactions, minsupp)

In [58]:
large_itemset

[['diary', 'ink'],
 ['diary', 'pen'],
 ['ink', 'pen'],
 ['ink', 'soap'],
 ['pen', 'soap']]

In [59]:
item_counts.update(item_count)

In [63]:
item_counts

{('diary',): 3,
 ('ink',): 3,
 ('pen',): 4,
 ('soap',): 2,
 ('book',): 1,
 ('diary', 'ink'): 2,
 ('diary', 'pen'): 3,
 ('diary', 'soap'): 1,
 ('ink', 'pen'): 3,
 ('ink', 'soap'): 2,
 ('pen', 'soap'): 2}

In [64]:
if len(large_itemset) != 0:
    k = k + 1
    large_itemsets[k] = large_itemset

In [65]:
prev_itemset = large_itemset

In [66]:
prev_itemset

[['diary', 'ink'],
 ['diary', 'pen'],
 ['ink', 'pen'],
 ['ink', 'soap'],
 ['pen', 'soap']]

In [67]:
candidate_itemset = apriori_gen(prev_itemset)

In [68]:
candidate_itemset

[['diary', 'ink', 'pen'], ['ink', 'pen', 'soap']]

In [70]:
pruned_itemset = prune(candidate_itemset, prev_itemset)

In [71]:
pruned_itemset

[['diary', 'ink', 'pen'], ['ink', 'pen', 'soap']]

In [72]:
large_itemset, item_count = get_lk_candidate(pruned_itemset, transactions, minsupp)

In [75]:
item_counts.update(item_count)

In [79]:
large_itemsets[k] = large_itemset

In [80]:
k = k+1

In [82]:
del large_itemsets[4]

In [83]:
large_itemsets

{1: [['diary'], ['ink'], ['pen'], ['soap']],
 2: [['diary', 'ink'],
  ['diary', 'pen'],
  ['ink', 'pen'],
  ['ink', 'soap'],
  ['pen', 'soap']],
 3: [['diary', 'ink', 'pen'], ['ink', 'pen', 'soap']]}

In [84]:
prev_itemset = large_itemset

In [85]:
candidate_itemset = apriori_gen(prev_itemset)

In [86]:
candidate_itemset

[]

In [88]:
k = 3

In [89]:
assoc_itemsets = get_assoc_itemsets(large_itemsets, minconf, item_counts, k)

IndexError: list index out of range

In [90]:
%debug

> <ipython-input-49-9724af5770e1>(68)get_assoc_itemsets()
     66             for j in range(k):
     67                 full = tuple(itemset)
---> 68                 left, right = itemset[:j] + itemset[j+1:], itemset[j]
     69                 assoc_itemset.add((tuple(full), tuple(left), right))
     70     for full, left, right in assoc_itemset:

ipdb> full
('diary', 'ink')
ipdb> k
3
ipdb> q


In [ ]:
['diary', 'ink', 'pen', 'soap']

In [90]:
from collections import defaultdict
from itertools import combinations

In [74]:
def get_l1_itemset(transactions, minsupp):
    result = []
    item_count = defaultdict(int)
    n_transaction = len(transactions)
    for transaction in transactions:
        for item in transaction:
            item_count[item] = item_count[item] + 1
    for key, value in item_count.items():
        if value / n_transaction >= minsupp:
            result.append([key])
    return sorted(result)

In [75]:
get_l1_itemset([[1,2,3,4], [1,2,3], [1,2], [1]], 0.50001)

[[1], [2]]

In [28]:
sorted([{2}, {1}])

[{2}, {1}]

In [31]:
def get_permutations(items):
    result = []
    n = len(items)
    for i in range(n-1):
        for j in range(i+1, n):
            result.append((items[i], items[j]))
    return result

In [ ]:
def get_prev_permutations(items):
    n = len(items):
        

In [92]:
list(combinations(range(5), 2))

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4)]

In [19]:
list(permutations([{1}, {2}, {3,4}], 2))

[({1}, {2}),
 ({1}, {3, 4}),
 ({2}, {1}),
 ({2}, {3, 4}),
 ({3, 4}, {1}),
 ({3, 4}, {2})]

In [38]:
{2,4}.union({3,2})

{2, 3, 4}

In [87]:
def apriori_gen(prev_itemsets):
    result = set()
    for item1, item2 in get_permutations(prev_itemsets):
        if item1[:-1] == item2[:-1]:
            if item1[-1] < item2[-1]:
                result.add(tuple(item1 + [item2[-1]]))
    result = list(result)
    return [list(x) for x in result]

In [97]:
apriori_gen([[1,2,3], [1,2,4], [1,3,4], [1,3,5], [2,3,4]])

[[1, 3, 4, 5], [1, 2, 3, 4]]

In [119]:

def apriori_gen(prev_itemsets):
    result = set()
    for item1, item2 in combinations(prev_itemsets, 2):
        if item1[:-1] == item2[:-1]:
            if item1[-1] < item2[-1]:
                result.add(tuple(item1 + [item2[-1]]))
    result = list(result)
    return sorted([list(x) for x in result])

In [126]:
apriori_gen([[1,2,3], [1,2,4], [1,3,4], [1,3,5], [2,3,4]])

[[1, 2, 3, 4], [1, 3, 4, 5]]

In [128]:
sorted([[14, 25, 46], [1, 22, 53], [7, 8, 9], [1,21,100]])

[[1, 21, 100], [1, 22, 53], [7, 8, 9], [14, 25, 46]]

In [130]:
list(range(2, 10))

[2, 3, 4, 5, 6, 7, 8, 9]

In [132]:
a = {}
a.update({1:1, 2:2})

In [134]:
a.update({(1,2) : 3, 'a': 3})

In [135]:
a

{1: 1, 2: 2, (1, 2): 3, 'a': 3}

In [146]:
result = set()
for itemset in [[1,2,3], [1,2,4], [1,3,4], [1,3,5], [2,3,4]]:
    for j in range(3):
        left, right = itemset[:j] + itemset[j+1:], itemset[j]
        result.add((tuple(left), right))

In [147]:
result

{((1, 2), 3),
 ((1, 2), 4),
 ((1, 3), 2),
 ((1, 3), 4),
 ((1, 3), 5),
 ((1, 4), 2),
 ((1, 4), 3),
 ((1, 5), 3),
 ((2, 3), 1),
 ((2, 3), 4),
 ((2, 4), 1),
 ((2, 4), 3),
 ((3, 4), 1),
 ((3, 4), 2),
 ((3, 5), 1)}

In [103]:
prune([[1, 3, 4, 5], [1, 2, 3, 4]], [[1,2,3], [1,2,4], [1,3,4], [1,3,5], [2,3,4]])

combination : (1, 4, 5), itemset : [[1, 2, 3], [1, 2, 4], [1, 3, 4], [1, 3, 5], [2, 3, 4]]
combination : (3, 4, 5), itemset : [[1, 2, 3], [1, 2, 4], [1, 3, 4], [1, 3, 5], [2, 3, 4]]


[[1, 2, 3, 4]]

In [148]:
item_count

NameError: name 'item_count' is not defined

In [149]:
def get_lk_candidate(pruned_itemset, transactions, minsupp):
    result = []
    item_count = {}
    k = len(pruned_itemset[0])
    for pruned in pruned_itemset:
        item_count[tuple(pruned)] = 0
    n_transaction = len(transactions)
    for transaction in transactions:
        if len(transaction) >= k:
            for comb in combinations(transaction, k):
                if comb in item_count:
                    item_count[comb] = item_count[comb] + 1
    for key, value in item_count.items():
        if value / n_transaction >= minsupp:
            result.append(sorted(list(key)))
    return sorted(result), item_count

In [150]:
get_lk_candidate([[1,2,3], [2,3,4]], [[1], [2], [1,2,3,4], [2,3,4], [2,3,4,5,6]], 0.25)

([[2, 3, 4]], {(1, 2, 3): 1, (2, 3, 4): 3})

In [118]:
list(combinations([1,2,3], 3))

[(1, 2, 3)]

In [166]:
from collections import defaultdict
from itertools import combinations


def get_l1_itemset(transactions, minsupp):
    result = []
    item_count = defaultdict(int)
    n_transaction = len(transactions)
    for transaction in transactions:
        for item in transaction:
            item_count[(item,)] = item_count[(item,)] + 1
    for key, value in item_count.items():
        if value / n_transaction >= minsupp:
            result.append([key])
    return sorted(result), item_count


def get_lk_candidate(pruned_itemset, transactions, minsupp):
    result = []
    item_count = {}
    k = len(pruned_itemset[0])
    for pruned in pruned_itemset:
        item_count[tuple(pruned)] = 0
    n_transaction = len(transactions)
    for transaction in transactions:
        if len(transaction) >= k:
            for comb in combinations(transaction, k):
                if comb in item_count:
                    item_count[comb] = item_count[comb] + 1
    for key, value in item_count.items():
        if value / n_transaction >= minsupp:
            result.append(sorted(list(key)))
    return sorted(result), item_count


def apriori_gen(prev_itemsets):
    result = set()
    for item1, item2 in combinations(prev_itemsets, 2):
        if item1[:-1] == item2[:-1]:
            if item1[-1] < item2[-1]:
                result.add(tuple(item1 + [item2[-1]]))
    result = list(result)
    return sorted([list(x) for x in result])


def prune(candidate_itemset, prev_itemset):
    k = len(prev_itemset[0]) + 1
    result = []
    for itemset in candidate_itemset:
        valid = True
        for combination in combinations(itemset, k-1):
            if list(combination) not in prev_itemset:
                valid = False
        if valid:
            result.append(itemset)
    return result


def get_assoc_itemsets(large_itemsets, minconf, item_counts, k):
    result = dict()
    assoc_itemsets = dict()
    assoc_itemset = set()
    for i in range(2, k+1):
        large_itemset = large_itemsets[i]
        for itemset in large_itemset:
            for j in range(k):
                full = tuple(itemset)
                left, right = itemset[:j] + itemset[j+1:], itemset[j]
                assoc_itemset.add((tuple(full), tuple(left), right))
    for full, left, right in assoc_itemset:
        conf = item_counts[left] / item_counts[full]
        assoc_itemsets[(left, right)] = conf
        if conf >= minconf:
            result[(left, right)] = conf
    return assoc_itemsets


def get_frequent_itemsets(transactions, minsupp, minconf):
    large_itemsets = dict()
    item_counts = dict()
    k = 1
    prev_itemset, item_count = get_l1_itemset(transactions, minsupp)
    item_counts.update(item_count)
    large_itemsets[k] = prev_itemset
    while len(prev_itemset) > 0:
        candidate_itemset = apriori_gen(prev_itemset)
        if len(candidate_itemset) == 0:
            break
        pruned_itemset = prune(candidate_itemset, prev_itemset)
        if len(pruned_itemset) == 0:
            break
        large_itemset, item_count = get_lk_candidate(pruned_itemset, transactions, minsupp)
        item_counts.update(item_count)
        if len(large_itemset) != 0:
            k = k + 1
            large_itemsets[k] = large_itemset
        else:
            break
        prev_itemset = large_itemset
    assoc_itemsets = get_assoc_itemsets(large_itemsets, minconf, item_counts, k)
    return large_itemsets, assoc_itemsets




In [169]:
large_itemsets = dict()
item_counts = dict()
k = 1
prev_itemset, item_count = get_l1_itemset(transactions, minsupp)

In [170]:
item_count

defaultdict(int, {('diary',): 3, ('ink',): 3, ('pen',): 4, ('soap',): 2})

In [171]:
item_counts.update(item_count)

In [172]:
item_counts

{('diary',): 3, ('ink',): 3, ('pen',): 4, ('soap',): 2}

In [173]:
large_itemsets[k] = prev_itemset

In [174]:
large_itemsets

{1: [[('diary',)], [('ink',)], [('pen',)]]}

In [179]:
list(combinations(prev_itemset, 2))

[([('diary',)], [('ink',)]),
 ([('diary',)], [('pen',)]),
 ([('ink',)], [('pen',)])]

In [175]:
candidate_itemset = apriori_gen(prev_itemset)

In [176]:
candidate_itemset

[[('diary',), ('ink',)], [('diary',), ('pen',)], [('ink',), ('pen',)]]

In [63]:
a = frozenset([1,2,3])

In [64]:
b = set(a)

In [68]:
b.pop()

1

In [71]:
({1, 2, 3} - {1, 2}).pop()

TypeError: int() argument must be a string, a bytes-like object or a number, not 'set'

In [66]:
b.add({4})
b.add({5})

TypeError: unhashable type: 'set'

In [62]:
b

{1, 2, 3, 4, 5}